In [1]:
# If you what use ngrams.py you should use python2
# Or, otherwise, you need to modify ngrams.py by yourself in order to use it in python3.
from __future__ import print_function
from __future__ import unicode_literals

import re
import numpy as np
from ngrams import ngrams
from collections import defaultdict

In [2]:
small_corpus = ['Why dont we start here',
                  'Why dont we end there',
                  'Let us start with a few other examples',
                  'We never start with an example with so few tokens',
                  'Tokens can be words that we start with in example docs'
                  ]

# HW3: Language Modeling
For this part of the assignment, you will implement two simple count-based n-gram language models: one based on maximum-likelihood estimation, and another based on Witten-Bell smoothing. The data you will be using is a subset of the Penn Treebank's tagged Wall Street Journal articles on which we have done some initial processing. There are two versions of the data for this assignment:

##### wsj.pos.gz
##### wsj-normalized.pos.gz
The difference is that, in the second (normalized) version of the data, we have collapsed some entries from certain tag categories (e.g. CDs, NNPs, etc.) into type-tokens to help reduce sparsity. Take a look at the data and see for yourself. Consider: what would be the benefits and drawbacks to this method of sparsity reduction? Note that, for this part of the assignment, the tags are un-necessary, so you'll want to work with the un-normalized version of the corpus.

https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html

### Task 1: produce a tag-free corpus

For this task, you have two jobs. 
* First, you need to write a function to filter out all tags. 
* Second, Make sure your code works for both wsj.pos.gz and wsj-normalized.pos.gz

####What to turn in
* your code
* some samples to show me that your code works as it should be

In [4]:
import gzip

#regex = re.compile(r"/[A-Z$,.]{1,4}") #stackoverflow says unnecessary, cached anyway
maxln = 1

def rem_tag(line):
    line = line.decode('utf-8')
    return re.sub(r"/[A-Z$,.]{1,4}",'',line)

print("######################## WSJ.POS Examples########################")
with gzip.open('wsj.pos.gz','r') as wsj:
    count = 0
    for line in wsj:
        line = rem_tag(line)
        print(line)
        count += 1
        if count > maxln:
            break

print("######################## WSJ.NORMALIZED Examples:########################")
with gzip.open('wsj-normalized.pos.gz','r') as wsj:
    count = 0
    for line in wsj:
        line = rem_tag(line)
        print(line)
        count += 1
        if count > maxln:
            break

######################## WSJ.POS Examples########################
Digital Equipment Corp. reported a 32 % decline in net income on a modest revenue gain in its fiscal first quarter , causing some analysts to predict weaker results ahead than they had expected .

Although the second-largest computer maker had prepared Wall Street for a poor quarter , analysts said they were troubled by signs of flat U.S. orders and a slowdown in the rate of gain in foreign orders . The Maynard , Mass. , company is in a transition in which it is trying to reduce its reliance on mid-range machines and establish a presence in workstations and mainframes .

######################## WSJ.NORMALIZED Examples:########################
<NNP> <NNP> <NNP> reported a <CD> % decline in net income on a modest revenue gain in its fiscal first quarter , causing some analysts to predict weaker results ahead than they had expected .

Although the second-largest computer maker had prepared <NNP> <NNP> for a poor quarter , 

<font color="red">Self assessment:</font>

### Maximum Likelihood
Now, start by producing code to compute maximum-likelihood estimate probabilities. Your code should be configurable with respect to the n-gram order- i.e., you should be able to set it to compute bigram, trigram, 4-gram, etc. probabilities. Refer to J&M and the lecture slides for definitions as needed. If you would like to write your own n-gram tokenization code, feel free to do so, but you may also use the ngrams.py utility class which contains a routine to take a list of tokens and produce a stream of n-grams with appropriate padding for the start and end of sentences.

#### Tip: 
* Start with a very small "toy" corpus of just a couple of sentences for debugging. 

* As discussed in class, I strongly recommend using nested defaultdicts as the foundational data structure for your language model, where the "outer" key is the prefix, and the value retrieved by that prefix is a second defaultdict  containing possible suffices for that prefix, each of which is an "inner" key. E.g., p("TRUTHS" | "HOLD THESE") would be retrieved by first looking up "HOLD THESE" and then from the resulting dictionary, looking up "TRUTHS": prob = trigrams[("HOLD","THESE")]["TRUTHS"] . Note that this arrangement makes it very easy to e.g. find out the number of times a given history occurs, the total probability mass assigned to all of a history's continuations, etc., all of which will be extremely helpful in the next part of the assignment.

* Use tuples to represent prefixes. E.g., instead of the string "HOLD THESE", use the tuple ("HOLD", "THESE"). Note that, in Python, lists are mutable, and therefore may not be used as keys in dictionaries- but tuples are immutable, and so make excellent keys.

* Don't forget about numerical underflow issues! You'll want to represent probabilities as negative base-2 log probabilities, and modify your arithmetic accordingly. I recommend experimenting with [the bitweight Python library](https://github.com/stevenbedrick/bitweight) (see its unit tests for example usage).
* 

#### What to turn in:
* your code 
* use your code to create a simple language model for small_corpus named as small_lm and show me that your output is correct(This is a small coupus so you could manully calculate the probalility).
* use your code to create language model for wsj.pos.gz named as wsj_lm

In [86]:
def make_dict(corpus, order):
    corpus_list = []
    corpus_dict = defaultdict(lambda: defaultdict(int))

    corpus_list += [corpus[i].lower().split() for i in range(len(corpus))]
    chunks = ngrams(corpus_list, order)
    lchunks = list(chunks)
    
    for i in range(len(lchunks)):
        corpus_dict[lchunks[i][0] ][lchunks[i][1]] += 1

    return corpus_dict

def mle(corp_dict, sentence):
    ...

corpus_dict = make_dict(small_corpus, 2)
print(corpus_dict[('start',)])
sum = 0
for key in corpus_dict[('start',)]:
    sum += corpus_dict[('start',)][key]
print(sum)

defaultdict(<class 'int'>, {'here': 1, 'with': 3})
4


<font color="red">Self assessment:</font>

### Smoothing

Once you’ve got an unsmoothed model working, move on to implementing Witten-Bell smoothing. Refer to the slides and J&M for details on how that ought to work.

#### Tip: 
* You can modify an already-populated defaultdict to change its default value (for example, to store a default backoff value for a particular history) by changing the object’s default_factory attribute. Consult the documentation for examples of how this works.
* As defined, W-B smoothing is highly recursive; you may find it more efficient to re-frame the algorithm in iterative terms.
* As in the previous section, start small.
* [This may offer you some help on how to implement Witten-Bell smoothing](http://www.ee.columbia.edu/~stanchen/e6884/labs/lab3/x207.html)


#### What to turn in:
* your code 
* use your code to create a simple smoothed language model based on small_lm  and show me that your output is correct(This is a small coupus so you could manully calculate the probalility).
* use your code to create a smoothed language model based on wsj_lm

<font color="red">Self assessment:</font>

### Evaluation via Perplexity
Explore the effects of n-gram order using perplexity. Perform ten-fold cross-validation on the WSJ corpus. On each iteration, this will give you a different 90/10 training/test split; train a smoothed language model on the 9 training sections, and compute the average per-token perplexity of the tenth section. The slides from the language modeling lecture give the equation for perplexity computation (as does J&M chapter 4); you'll need to modify the equation a bit, since we're using log-probabilities. 

Now, try this for unigram, bigram, trigram, and 4-gram models. 

#### What to turn in
* your cross-validation function. You are not suppose to use any cross-validation function from any module. You should implement it by yourself.
* your perplexity function
* cross-validation result for unigram, bigram, trigram, and 4-gram models on wsj.pos.gz
* cross-validation result for unigram, bigram, trigram, and 4-gram models on wsj-normalized.pos.gz.
* Answer following 2 questions: 
    * How does perplexity change as the model order size increases?
    * How does perplexity change as the data changed?

<font color="red">Self assessment:</font>